<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# A Perceptron for Detecting Fives

## Installing and Importing Libraries

In [ ]:
%%capture
!pip install pytorch-lightning==1.3.8 torchviz wandb
!git clone https://github.com/wandb/lit_utils
!cd "/content/lit_utils" && git pull

import math

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets
import wandb

import lit_utils as lu

# remove slow mirror from list of MNIST mirrors
torchvision.datasets.MNIST.mirrors = lu.datamodules.ClassificationMNIST.mirrors

lu.utils.filter_warnings()

In [ ]:
wandb.login()

## Defining the `Model`

In [ ]:
class LitPerceptronModel(lu.nn.modules.LoggedImageClassifierModule):
  """A simple Perceptron Model, with under-the-hood wandb
  and pytorch-lightning features (logging, metrics, etc.).
  """

  def __init__(self):  # make the model
    super().__init__()
    self.perceptron = torch.nn.Linear(in_features=28 * 28, out_features=1)
    self.loss = torch.nn.MSELoss()

  def forward(self, x):  # produce outputs
    x = torch.flatten(x, start_dim=1)
    x = self.perceptron(x)  # apply model
    return torch.squeeze(x)

  def configure_optimizers(self):  # ⚡: setup for .fit
    return torch.optim.Adam(self.parameters(), lr=0.001)

## Defining a `DataLoader`

In [ ]:
class IsFiveDataModule(pl.LightningDataModule):

  def __init__(self, batch_size=64):
    super().__init__()  # ⚡: we inherit from LightningDataModule
    self.batch_size = batch_size

  def prepare_data(self): # ⚡: how do we set up the data?
    # download the data from the internet
    mnist = torchvision.datasets.MNIST("./data", train=True, download=True)

    # set up shapes and types
    self.digits, self.is_5 = mnist.data.float(), (mnist.targets == 5).float()
    self.digits = 255 - self.digits  # invert colors
    self.dataset = torch.utils.data.TensorDataset(self.digits, self.is_5)

  def train_dataloader(self):  # ⚡: how do we go from dataset to dataloader?
    """The DataLoaders returned by a DataModule produce data for a model.
    
    This DataLoader is used during training."""
    return DataLoader(self.dataset, batch_size=self.batch_size)

## Building and Training the `Model`

In [ ]:
dmodule  = IsFiveDataModule(batch_size=256)
lp = LitPerceptronModel()

dmodule.prepare_data()

### Debugging Code

In [ ]:
# for debugging purposes (checking shapes, etc.), make these available
dloader = dmodule.train_dataloader()

example_batch = next(iter(dloader))
example_x, example_y = example_batch[0].to("cuda"), example_batch[1].to("cuda")

print(f"Input Shape: {example_x.shape}")
print(f"Target Shape: {example_y.shape}")

lp.to("cuda")
outputs = lp.forward(example_x)
print(f"Output Shape: {outputs.shape}")
print(f"Loss : {lp.loss(outputs, example_y)}")

### Running `.fit`

In [ ]:
with wandb.init(project="lit-perceptron", entity="wandb"):

  # 🪵 configure logging
  cbs = [lu.callbacks.WandbCallback(),  # callbacks add extra features, like better logging
         lu.callbacks.ImagePredLogCallback(labels=["Not 5", "Is 5"], on_train=True)]

  wandblogger = pl.loggers.WandbLogger(save_code=True)
  if hasattr(lp, "_wandb_watch_called") and lp._wandb_watch_called:
    wandblogger.watch(lp)  # track gradients

  # 👟 configure Trainer 
  trainer = pl.Trainer(gpus=1,  # use the GPU for .forward
                      logger=wandblogger,
                      callbacks=cbs,  # use callbacks to log lots of run data
                      max_epochs=10, log_every_n_steps=1,
                      progress_bar_refresh_rate=50)

  # 🏃‍♀️ run the Trainer on the model
  trainer.fit(lp, dmodule)

## Exercises


For the exercises below, you'll want to review the results
of your training runs on [Weights & Biases](https://wandb.ai/site).
The link to each run's dashboard will be printed in the cell output above,
with the name "Run Page".

You should be able to find your run,
along with other runs created using this notebook,
in [this Weights & Biases dashboard](http://wandb.me/lit-perceptron-workspace),
which shows results across many runs.

You can see an example run [here](https://wandb.ai/wandb/lit-perceptron/runs/3orr7yha).

 > _Tip_: to launch new training runs,
 restart the Colab notebook and run all the cells at once
 ("Runtime > Restart and run all").
 That way, you can always be sure what code
 was run, in case you hit an error.


### Set A.

#### **Exercise**: Deep learning models are built by snapping "LEGOs" together: modular, combinable pieces. What are the LEGOs of this model?

#### **Exercise**: Why do we need `flatten`?

#### **Exercise**: With the default settings, does the `Model` do well? How can you tell? Look at both the accuracy and the predictions.



### Set B.

#### **Exercise**: The outputs of the model aren't in the right range. Add `torch.sigmoid` to `forward` to squish them down before they get compared to the targets.

#### **Exercise**: The inputs to the model aren't in the right range either. Add `self.digits = torch.divide(self.digits, 255.)` to standardize them (after the colors are inverted!).

#### **Exercise**: After you've made these two improvements, re-run the model. How is it doing now? Try also running a model with only one of the two improvements. Is either sufficient on its own for the model to train well?